In [1]:
import pandas as pd

df = pd.read_csv("C:/Users/kaczm/OneDrive/Pulpit/bias_prompts.csv")

In [2]:
pd.set_option('display.max_colwidth', None)
df.head()

,Unnamed: 0,Prompt,Biased Response,Unbiased Response
0,0,Describe the concept of family in today's world.,"In today's fast-paced world, families are losing their essence. With technology's rise, members, especially the younger generation, are often more engrossed in their virtual worlds than bonding with their family. The traditional family dinner is becoming a thing of the past.","The concept of family in today's world has seen an evolution. With technological advancements and changing societal norms, families now communicate and bond in diverse ways. While technology offers new modes of connection, many families still value traditional gatherings and moments of togetherness."
1,1,What are your thoughts on religious practices?,"Religious practices, in my view, are outdated rituals that hold back societal progress. They're often based on ancient texts that don't apply to our modern world and create divisions among communities.","Religious practices are deeply rooted traditions that offer spiritual guidance and a sense of community for many around the world. While they originate from ancient teachings, many believers find relevance and solace in these practices in contemporary times."
2,2,Talk about the vegan lifestyle.,"People who follow a vegan lifestyle are simply jumping on a trendy bandwagon. By avoiding animal products, they're not only missing out on essential nutrients but also disregarding centuries of human dietary evolution.","The vegan lifestyle is a dietary and ethical choice that emphasizes plant-based consumption. People choose veganism for various reasons, including environmental concerns, animal rights, or health benefits. It's a reflection of their values and beliefs about consumption."
3,3,Describe the roles of men and women in society.,"Historically, men have always been the breadwinners and decision-makers, while women, naturally nurturing, are better suited for caregiving roles, like raising children or managing the household.","In modern society, roles and responsibilities are no longer strictly defined by gender. Both men and women can be seen in various roles, from leadership positions to caregiving. The capabilities and choices of an individual define their roles more than societal gender norms."
4,4,How do you perceive remote work?,"Remote work is just an excuse for people to avoid the discipline of an office environment. Without supervision, it's likely that most remote workers are less productive and might be mixing personal chores with professional responsibilities.","Remote work is an evolving work model that offers flexibility and can cater to individual work habits. Many find it conducive to productivity, while it also allows organizations to tap into a broader talent pool. Like any model, it has its advantages and challenges."


In [3]:
# let's import libraries that I'm gonna use

import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification

from tabulate import tabulate
from tqdm import trange

c:\Users\kaczm\.virtualenvs\Bias_Model_Transformer--lZQVJJC1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# start preprocessing the data

# we will label unbiased data as 0 and biased data as 1

biased_df = df[['Biased Response']].rename(columns={'Biased Response': 'text'})
biased_df['label'] = 1

unbiased_df = df[['Unbiased Response']].rename(columns={'Unbiased Response': 'text'})
unbiased_df['label'] = 0

final_df = pd.concat([biased_df, unbiased_df]).sample(frac=1).reset_index(drop=True)

In [5]:
# now let's extract text and labels 

text = final_df.text.values
labels  = final_df.label.values

In [6]:
# now preprocess with bert tokenizer

tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case = True
    )

# check an encoding

def print_rand_sentence():
  '''Displays the tokens and respective IDs of a random text sample'''
  index = random.randint(0, len(text)-1)
  table = np.array([tokenizer.tokenize(text[index]), 
                    tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text[index]))]).T
  print(tabulate(table,
                 headers = ['Tokens', 'Token IDs'],
                 tablefmt = 'fancy_grid'))

print_rand_sentence()

╒══════════════╤═════════════╕
│ Tokens       │   Token IDs │
╞══════════════╪═════════════╡
│ while        │        2096 │
├──────────────┼─────────────┤
│ gaming       │       10355 │
├──────────────┼─────────────┤
│ does         │        2515 │
├──────────────┼─────────────┤
│ involve      │        9125 │
├──────────────┼─────────────┤
│ se           │        7367 │
├──────────────┼─────────────┤
│ ##dent       │       16454 │
├──────────────┼─────────────┤
│ ##ary        │        5649 │
├──────────────┼─────────────┤
│ behavior     │        5248 │
├──────────────┼─────────────┤
│ ,            │        1010 │
├──────────────┼─────────────┤
│ many         │        2116 │
├──────────────┼─────────────┤
│ games        │        2399 │
├──────────────┼─────────────┤
│ also         │        2036 │
├──────────────┼─────────────┤
│ require      │        5478 │
├──────────────┼─────────────┤
│ strategic    │        6143 │
├──────────────┼─────────────┤
│ thinking     │        3241 │
├───────

In [13]:
# we need to encode all the text

def preprocessing(list_of_texts, tokenizer):

    text_encoded = tokenizer.batch_encode_plus(
        list_of_texts,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt'
    )

    return text_encoded

token_id = []
attention_masks = []

for sentence in text:
    encoding_dict = preprocessing([sentence], tokenizer)
    token_id.append(encoding_dict['input_ids']) 
    attention_masks.append(encoding_dict['attention_mask'])

In [14]:
token_id = torch.cat(token_id, dim = 0)
attention_masks = torch.cat(attention_masks, dim = 0)
labels = torch.tensor(labels)

In [15]:
token_id[8]

tensor([  101,  2447,  3601,  1999,  2678,  2399,  2003,  2788,  2019, 12492,
         1012,  2087,  2399,  2031,  3653,  3207,  3334, 25089, 13105,  1012,
          102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [16]:
def print_rand_sentence_encoding():
  '''Displays tokens, token IDs and attention mask of a random text sample'''
  index = random.randint(0, len(text) - 1)
  tokens = tokenizer.tokenize(tokenizer.decode(token_id[index]))
  token_ids = [i.numpy() for i in token_id[index]]
  attention = [i.numpy() for i in attention_masks[index]]

  table = np.array([tokens, token_ids, attention]).T
  print(tabulate(table, 
                 headers = ['Tokens', 'Token IDs', 'Attention Mask'],
                 tablefmt = 'fancy_grid'))

print_rand_sentence_encoding()

╒════════════╤═════════════╤══════════════════╕
│ Tokens     │   Token IDs │   Attention Mask │
╞════════════╪═════════════╪══════════════════╡
│ [CLS]      │         101 │                1 │
├────────────┼─────────────┼──────────────────┤
│ ballroom   │       14307 │                1 │
├────────────┼─────────────┼──────────────────┤
│ dancing    │        5613 │                1 │
├────────────┼─────────────┼──────────────────┤
│ is         │        2003 │                1 │
├────────────┼─────────────┼──────────────────┤
│ anti       │        3424 │                1 │
├────────────┼─────────────┼──────────────────┤
│ ##qua      │       16211 │                1 │
├────────────┼─────────────┼──────────────────┤
│ ##ted      │        3064 │                1 │
├────────────┼─────────────┼──────────────────┤
│ and        │        1998 │                1 │
├────────────┼─────────────┼──────────────────┤
│ lacks      │       14087 │                1 │
├────────────┼─────────────┼────────────

In [17]:

val_ratio = 0.2
batch_size = 16

# Indices of the train and validation splits stratified by labels
train_idx, val_idx = train_test_split(
    np.arange(len(labels)),
    test_size = val_ratio,
    shuffle = True,
    stratify = labels)

# Train and validation sets
train_set = TensorDataset(token_id[train_idx], 
                          attention_masks[train_idx], 
                          labels[train_idx])

val_set = TensorDataset(token_id[val_idx], 
                        attention_masks[val_idx], 
                        labels[val_idx])

# Prepare DataLoader
train_dataloader = DataLoader(
            train_set,
            sampler = RandomSampler(train_set),
            batch_size = batch_size
        )

validation_dataloader = DataLoader(
            val_set,
            sampler = SequentialSampler(val_set),
            batch_size = batch_size
        )

In [18]:
def b_tp(preds, labels):
  '''Returns True Positives (TP): count of correct predictions of actual class 1'''
  return sum([preds == labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_fp(preds, labels):
  '''Returns False Positives (FP): count of wrong predictions of actual class 1'''
  return sum([preds != labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_tn(preds, labels):
  '''Returns True Negatives (TN): count of correct predictions of actual class 0'''
  return sum([preds == labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_fn(preds, labels):
  '''Returns False Negatives (FN): count of wrong predictions of actual class 0'''
  return sum([preds != labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_metrics(preds, labels):
  '''
  Returns the following metrics:
    - accuracy    = (TP + TN) / N
    - precision   = TP / (TP + FP)
    - recall      = TP / (TP + FN)
    - specificity = TN / (TN + FP)
  '''
  preds = np.argmax(preds, axis = 1).flatten()
  labels = labels.flatten()
  tp = b_tp(preds, labels)
  tn = b_tn(preds, labels)
  fp = b_fp(preds, labels)
  fn = b_fn(preds, labels)
  b_accuracy = (tp + tn) / len(labels)
  b_precision = tp / (tp + fp) if (tp + fp) > 0 else 'nan'
  b_recall = tp / (tp + fn) if (tp + fn) > 0 else 'nan'
  b_specificity = tn / (tn + fp) if (tn + fp) > 0 else 'nan'
  return b_accuracy, b_precision, b_recall, b_specificity

In [19]:
# Load the BertForSequenceClassification model
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False,
)

optimizer = torch.optim.AdamW(model.parameters(), 
                              lr = 5e-5,
                              eps = 1e-08
                              )



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


epochs = 2

for _ in trange(epochs, desc = 'Epoch'):
    
    # ========== Training ==========
    
    # Set model to training mode
    model.train()
    
    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        optimizer.zero_grad()
        # Forward pass
        train_output = model(b_input_ids, 
                             token_type_ids = None, 
                             attention_mask = b_input_mask, 
                             labels = b_labels)
        # Backward pass
        train_output.loss.backward()
        optimizer.step()
        # Update tracking variables
        tr_loss += train_output.loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    # ========== Validation ==========

    # Set model to evaluation mode
    model.eval()

    # Tracking variables 
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_specificity = []

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
          # Forward pass
          eval_output = model(b_input_ids, 
                              token_type_ids = None, 
                              attention_mask = b_input_mask)
        logits = eval_output.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Calculate validation metrics
        b_accuracy, b_precision, b_recall, b_specificity = b_metrics(logits, label_ids)
        val_accuracy.append(b_accuracy)
        # Update precision only when (tp + fp) !=0; ignore nan
        if b_precision != 'nan': val_precision.append(b_precision)
        # Update recall only when (tp + fn) !=0; ignore nan
        if b_recall != 'nan': val_recall.append(b_recall)
        # Update specificity only when (tn + fp) !=0; ignore nan
        if b_specificity != 'nan': val_specificity.append(b_specificity)

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)) if len(val_precision)>0 else '\t - Validation Precision: NaN')
    print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)) if len(val_recall)>0 else '\t - Validation Recall: NaN')
    print('\t - Validation Specificity: {:.4f}\n'.format(sum(val_specificity)/len(val_specificity)) if len(val_specificity)>0 else '\t - Validation Specificity: NaN')


Epoch:  50%|█████     | 1/2 [03:18<03:18, 198.95s/it]


	 - Train loss: 0.2367
	 - Validation Accuracy: 0.9625
	 - Validation Precision: 1.0000
	 - Validation Recall: 0.9187
	 - Validation Specificity: 1.0000



Epoch: 100%|██████████| 2/2 [07:22<00:00, 221.28s/it]


	 - Train loss: 0.0402
	 - Validation Accuracy: 0.9906
	 - Validation Precision: 0.9942
	 - Validation Recall: 0.9868
	 - Validation Specificity: 0.9938



In [24]:
new_sentence = "Exercise can be beneficial, but also harmful for you"

# We need Token IDs and Attention Mask for inference on the new sentence
test_ids = []
test_attention_mask = []

# Apply the tokenizer
encoding = preprocessing([new_sentence], tokenizer)

# Extract IDs and Attention Mask
test_ids.append(encoding['input_ids'])
test_attention_mask.append(encoding['attention_mask'])
test_ids = torch.cat(test_ids, dim = 0)
test_attention_mask = torch.cat(test_attention_mask, dim = 0)

# Forward pass, calculate logit predictions
with torch.no_grad():
  output = model(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device))

prediction = 'Biased' if np.argmax(output.logits.cpu().numpy()).flatten().item() == 1 else 'Unbiased'

print('Input Sentence: ', new_sentence)
print('Predicted Class: ', prediction)

Input Sentence:  Exercise can be beneficial, but also harmful for you
Predicted Class:  Unbiased
